In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import onnx
from onnx import helper
from onnx import AttributeProto, TensorProto, GraphProto

In [2]:
# Create one input (ValueInfoProto)
X = helper.make_tensor_value_info('X', TensorProto.FLOAT, [1, 2])

# Create one output (ValueInfoProto)
Y = helper.make_tensor_value_info('Y', TensorProto.FLOAT, [1, 4])

# Create a node (NodeProto)
node_def = helper.make_node(
    'Pad', # node name
    ['X'], # inputs
    ['Y'], # outputs
    mode='constant', # Attributes
    value=1.5,
    pads=[0, 1, 0, 1],
)

# Create the graph (GraphProto)
graph_def = helper.make_graph(
    [node_def],
    "test-model",
    [X],
    [Y],
)

# Create the model (ModelProto)
model_def = helper.make_model(graph_def,
                              producer_name='onnx-example')

print('The ir_version in model: {}\n'.format(model_def.ir_version))
print('The producer_name in model: {}\n'.format(model_def.producer_name))
print('The graph in model:\n{}'.format(model_def.graph))
onnx.checker.check_model(model_def)
print('The model is checked!')

The ir_version in model: 5

The producer_name in model: onnx-example

The graph in model:
node {
  input: "X"
  output: "Y"
  op_type: "Pad"
  attribute {
    name: "mode"
    s: "constant"
    type: STRING
  }
  attribute {
    name: "pads"
    ints: 0
    ints: 1
    ints: 0
    ints: 1
    type: INTS
  }
  attribute {
    name: "value"
    f: 1.5
    type: FLOAT
  }
}
name: "test-model"
input {
  name: "X"
  type {
    tensor_type {
      elem_type: 1
      shape {
        dim {
          dim_value: 1
        }
        dim {
          dim_value: 2
        }
      }
    }
  }
}
output {
  name: "Y"
  type {
    tensor_type {
      elem_type: 1
      shape {
        dim {
          dim_value: 1
        }
        dim {
          dim_value: 4
        }
      }
    }
  }
}

The model is checked!


In [3]:
# Save onnx model

model_path = 'C:/Users/t-agkum/onnxruntime/onnxruntime/python/tools/quantization/models/example.onnx'
onnx.save(model_def, model_path)

In [8]:
# Adding a node (modifying base_model)

base_model = onnx.load(model_path) # loading model (X-Pad-Y)

#print(base_model)

input_name = 'X'
reduce_min_name = input_name + "_ReduceMin"
reduce_min_node = onnx.helper.make_node("ReduceMin", [input_name],
                    [reduce_min_name + ":0"], reduce_min_name, keepdims=0)

# Create the graph (GraphProto)
graph_def = helper.make_graph(
    [reduce_min_node],
    "test-model",
    [X],
    [Y],
)

# Create the model (ModelProto)
model_def_new = helper.make_model(graph_def)

# print(model_def_new)

onnx.save(model_def_new, 'C:/Users/t-agkum/onnxruntime/onnxruntime/python/tools/quantization/models/example_modified.onnx')

ir_version: 5
graph {
  node {
    input: "X"
    output: "X_ReduceMin:0"
    name: "X_ReduceMin"
    op_type: "ReduceMin"
    attribute {
      name: "keepdims"
      i: 0
      type: INT
    }
  }
  name: "test-model"
  input {
    name: "X"
    type {
      tensor_type {
        elem_type: 1
        shape {
          dim {
            dim_value: 1
          }
          dim {
            dim_value: 2
          }
        }
      }
    }
  }
  output {
    name: "Y"
    type {
      tensor_type {
        elem_type: 1
        shape {
          dim {
            dim_value: 1
          }
          dim {
            dim_value: 4
          }
        }
      }
    }
  }
}
opset_import {
  version: 10
}



In [10]:
# Adding a node (modifying base_model)

base_model = onnx.load(model_path) # loading model (X-Pad-Y)

#print(base_model)

input_name = 'X'
reduce_min_name = input_name + "_ReduceMin"
reduce_min_node = onnx.helper.make_node("ReduceMin", [input_name],
                    [reduce_min_name + ":0"], reduce_min_name, keepdims=0)

# Create the graph (GraphProto)
graph_def = helper.make_graph(
    [reduce_min_node],
    "test-model",
    [X],
    [Y],
)

print(graph_def.node)

# Create the model (ModelProto)
model_def_new = helper.make_model(graph_def)

# print(model_def_new)

onnx.save(model_def_new, 'C:/Users/t-agkum/onnxruntime/onnxruntime/python/tools/quantization/models/example_modified.onnx')

[input: "X"
output: "X_ReduceMin:0"
name: "X_ReduceMin"
op_type: "ReduceMin"
attribute {
  name: "keepdims"
  i: 0
  type: INT
}
]


In [12]:
print(base_model.graph)

node {
  input: "X"
  output: "Y"
  op_type: "Pad"
  attribute {
    name: "mode"
    s: "constant"
    type: STRING
  }
  attribute {
    name: "pads"
    ints: 0
    ints: 1
    ints: 0
    ints: 1
    type: INTS
  }
  attribute {
    name: "value"
    f: 1.5
    type: FLOAT
  }
}
name: "test-model"
input {
  name: "X"
  type {
    tensor_type {
      elem_type: 1
      shape {
        dim {
          dim_value: 1
        }
        dim {
          dim_value: 2
        }
      }
    }
  }
}
output {
  name: "Y"
  type {
    tensor_type {
      elem_type: 1
      shape {
        dim {
          dim_value: 1
        }
        dim {
          dim_value: 4
        }
      }
    }
  }
}



In [ ]:
# Adding nodes to mobilenet model

